<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Sprint Challenge
## *Data Science Unit 4 Sprint 1*

After a week of Natural Language Processing, you've learned some cool new stuff: how to process text, how turn text into vectors, and how to model topics from documents. Apply your newly acquired skills to one of the most famous NLP datasets out there: [Yelp](https://www.yelp.com/dataset/challenge). As part of the job selection process, some of my friends have been asked to create analysis of this dataset, so I want to empower you to have a head start.  

The real dataset is massive (almost 8 gigs uncompressed). I've sampled the data for you to something more managable for the Sprint Challenge. You can analyze the full dataset as a stretch goal or after the sprint challenge. As you work on the challenge, I suggest adding notes about your findings and things you want to analyze in the future.

## Challenge Objectives
*Successfully complete these all these objectives to earn a 2. There are more details on each objective further down in the notebook.*
* <a href="#p1">Part 1</a>: Write a function to tokenize the yelp reviews
* <a href="#p2">Part 2</a>: Create a vector representation of those tokens
* <a href="#p3">Part 3</a>: Use your tokens in a classification model on yelp rating
* <a href="#p4">Part 4</a>: Estimate & Interpret a topic model of the Yelp reviews

In [29]:
# imports
import numpy as np
import gensim
import os
import re

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora

from gensim.models.ldamulticore import LdaMulticore

import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [13]:
import pandas as pd

yelp = pd.read_json('review_sample.json', lines=True)

In [14]:
yelp.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",10,n1LM36qNg4rqGXIcvVXv8w
1,eMYeEapscbKNqUDCx705hg,0,2015-12-16 05:31:03,0,DoQDWJsNbU0KL1O29l_Xug,4,Came here for lunch Togo. Service was quick. S...,0,5CgjjDAic2-FAvCtiHpytA
2,6Q7-wkCPc1KF75jZLOTcMw,1,2010-06-20 19:14:48,1,DDOdGU7zh56yQHmUnL1idQ,3,I've been to Vegas dozens of times and had nev...,2,BdV-cf3LScmb8kZ7iiBcMA
3,k3zrItO4l9hwfLRwHBDc9w,3,2010-07-13 00:33:45,4,LfTMUWnfGFMOfOIyJcwLVA,1,We went here on a night where they closed off ...,5,cZZnBqh4gAEy4CdNvJailQ
4,6hpfRwGlOzbNv7k5eP9rsQ,1,2018-06-30 02:30:01,0,zJSUdI7bJ8PNJAg4lnl_Gg,4,"3.5 to 4 stars\n\nNot bad for the price, $12.9...",5,n9QO4ClYAS7h9fpQwa5bhA


## Part 1: Tokenize Function
<a id="#p1"></a>

Complete the function `tokenize`. Your function should
- accept one document at a time
- return a list of tokens

You are free to use any method you have learned this week.

In [16]:
# define stop words

STOPWRODS = set(STOPWORDS)
len(STOPWORDS)

337

In [17]:
def tokenize(doc):
    
    return [token for token in simple_preprocess(doc) if token not in STOPWORDS]

In [18]:
# create a list of the reviews to tokenize
yelp['text_tokens'] = yelp['text'].apply(tokenize)
yelp.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id,text_tokens
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",10,n1LM36qNg4rqGXIcvVXv8w,"[beware, fake, fake, fake, small, business, lo..."
1,eMYeEapscbKNqUDCx705hg,0,2015-12-16 05:31:03,0,DoQDWJsNbU0KL1O29l_Xug,4,Came here for lunch Togo. Service was quick. S...,0,5CgjjDAic2-FAvCtiHpytA,"[came, lunch, togo, service, quick, staff, fri..."
2,6Q7-wkCPc1KF75jZLOTcMw,1,2010-06-20 19:14:48,1,DDOdGU7zh56yQHmUnL1idQ,3,I've been to Vegas dozens of times and had nev...,2,BdV-cf3LScmb8kZ7iiBcMA,"[ve, vegas, dozens, times, stepped, foot, circ..."
3,k3zrItO4l9hwfLRwHBDc9w,3,2010-07-13 00:33:45,4,LfTMUWnfGFMOfOIyJcwLVA,1,We went here on a night where they closed off ...,5,cZZnBqh4gAEy4CdNvJailQ,"[went, night, closed, street, party, best, act..."
4,6hpfRwGlOzbNv7k5eP9rsQ,1,2018-06-30 02:30:01,0,zJSUdI7bJ8PNJAg4lnl_Gg,4,"3.5 to 4 stars\n\nNot bad for the price, $12.9...",5,n9QO4ClYAS7h9fpQwa5bhA,"[stars, bad, price, lunch, seniors, pay, eatin..."


## Part 2: Vector Representation
<a id="#p2"></a>
1. Create a vector representation of the reviews
2. Write a fake review and query for the 10 most similiar reviews, print the text of the reviews. Do you notice any patterns?
    - Given the size of the dataset, it will probably be best to use a `NearestNeighbors` model for this. 

In [20]:
# create a tfidf feature matrix
tfidf = TfidfVectorizer(stop_words = STOPWORDS,
                       ngram_range = (1,2),
                       max_df = .97,
                       min_df = 3,
                       max_features = 1000)

# create a vocab and get word counts
dtm = tfidf.fit_transform(yelp['text'])

# get feature names to use as dataframe column headers
dtm = pd.DataFrame(dtm.todense(), columns = tfidf.get_feature_names())

# view feature matrix as df
dtm.head()

,00,10,10 minutes,100,11,12,15,15 minutes,20,20 minutes,...,write,wrong,year,years,years ago,yelp,yes,young,yum,yummy
0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.166002,0.211222,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.121427,0.0,0.0,0.0,0.0,0.153053,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.170251,0.152531,0.0,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


In [25]:
fake = ['This is the best food I have ever tasted!!! A definite must for anyone who wants to be truly happy!']

In [23]:
# fit a nn model on dtm
nn = NearestNeighbors(n_neighbors = 10, algorithm = 'kd_tree')
nn.fit(dtm)

NearestNeighbors(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                 radius=1.0)

In [26]:
# query using nn
# turn the review into a tf idf vector
my_review = tfidf.transform(fake)
my_review

<1x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 5 stored elements in Compressed Sparse Row format>

In [27]:
# turn the tf idf vector into a dense matrix
# and plug it into the nn model
nn.kneighbors(my_review.todense())

(array([[1.        , 1.        , 1.12855897, 1.13796486, 1.17175736,
         1.17396907, 1.17993207, 1.1826381 , 1.18401837, 1.18527893]]),
 array([[6311, 6204,  698, 5829, 4162, 9653, 2389, 8376, 4336, 2451]]))

In [28]:
# print the reviews that are most similar
indices = [6311, 6204,  698, 5829, 4162, 9653, 2389, 8376, 4336, 2451]
counter = 1
for index in indices:
    print(yelp['text'][index])
    print('________________________________________________________________')
    print(counter)
    counter = counter + 1

天氣很熱吃不下東西，今天我點了一個韓國冷面湯、餐後點了甜點，冰沙系列不會太甜膩，覺得店家很用心製作，包含擺盤精緻、佐料衛生，夏日想開胃，這是一個不錯的選擇，服務人員也很敬業，以後會常常來
________________________________________________________________
1
旅行でラスベガスに来ましたがネイルがはげてるのが気になり、探したお店でした。
質問にも丁寧に答えてくれましたし、日本人の方も日本語が話せる方も居て、とても綺麗で居心地のいいお店でした。 
ネイルはちはるさんと言う綺麗な方が丁寧にしてくれとても気に入りました。
予定になかったまつ毛エクステもお願いし、日本ではまだあまりないブラウンカラーのエクステをしてもらい、とても気に入りました。
また是非マッサージなどで伺いたいと思います。
________________________________________________________________
2
Amazing job ! I'm truly happy with my service me and my sister did our fullset and pedicure! My go to place now ! Thank you Hana & Min !!!!!
________________________________________________________________
3
This place is amazing! It was my second time going and Marie never fails to make me happy. She's so nice and sweet & she knows how to do her job well. This place truly makes you feel like a princess it's comfortable to be & very professional. I would truly recommend anyone to this place and would go back. My lashes look amazing
___________________

## Part 3: Classification
<a id="#p3"></a>
Your goal in this section will be to predict `stars` from the review dataset. 

1. Create a piepline object with a sklearn `CountVectorizer` or `TfidfVector` and any sklearn classifier. Use that pipeline to estimate a model to predict `stars`. Use the Pipeline to predict a star rating for your fake review from Part 2. 
2. Tune the entire pipeline with a GridSearch

In [31]:
# define pipeline components
vect = TfidfVectorizer(stop_words = STOPWORDS, ngram_range = (1,2))
clf = RandomForestClassifier()

pipe = Pipeline([('vect', vect), ('clf', clf)])

In [34]:
# define the search space and look for the best hyperparameters of the vectorizer
parameters = {
            'vect__max_df': (.75, 1.0),
            'vect__min_df': (.02, .05),
            'vect__max_features': (500, 2000),
            'clf__n_estimators': (5,20),
            'clf__max_depth': (15, 30)
}

grid_search = GridSearchCV(pipe, parameters, cv = 5, n_jobs = -1, verbose = 1)
grid_search.fit(yelp['text'], yelp['stars'])

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   24.8s
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:  1.9min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 2),
                                                        no

In [35]:
# Predict on my own reviews
good_review = fake
bad_review = ['The food was TERRIBLE!!! I found a bug in my burrito and when I told the manager he just laughed and said it was a good source of protein']
pred = grid_search.predict(good_review)
pred2 = grid_search.predict(bad_review)
print(pred, pred2)

[5] [1]


## Part 4: Topic Modeling

Let's find out what those yelp reviews are saying! :D

1. Estimate a LDA topic model of the review text
    - Keep the `iterations` parameter at or below 5 to reduce run time
    - The `workers` parameter should match the number of physical cores on your machine.
2. Create 1-2 visualizations of the results
    - You can use the most important 3 words of a topic in relevant visualizations. Refer to yesterday's notebook to extract. 
3. In markdown, write 1-2 paragraphs of analysis on the results of your topic model

__*Note*__: You can pass the DataFrame column of text reviews to gensim. You do not have to use a generator.

In [36]:
from gensim.models import LdaMulticore
from gensim.corpora import Dictionary

Learn the vocubalary of the yelp data:

In [38]:
id2word = Dictionary(yelp['text_tokens'])

Create a bag of words representation of the entire corpus

In [39]:
corpus = [id2word.doc2bow(text) for text in yelp['text_tokens']]

Your LDA model should be ready for estimation: 

In [40]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   iterations=5,
                   workers=4,
                   num_topics = 10 # You can change this parameter
                  )

Create 1-2 visualizations of the results

In [41]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

pyLDAvis.gensim.prepare(lda, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.005499 -0.000860       1        1  10.772785
7     -0.001079  0.003835       2        1  10.500250
1     -0.001802 -0.002217       3        1  10.449278
9     -0.000277  0.003304       4        1  10.177107
0     -0.000585  0.000484       5        1   9.833835
2      0.001674  0.001873       6        1   9.806819
4     -0.000749 -0.004675       7        1   9.746191
6      0.003349 -0.001791       8        1   9.628868
8      0.000498  0.001068       9        1   9.628693
5      0.004468 -0.001021      10        1   9.456177, topic_info=          Term         Freq        Total Category  logprob  loglift
40        good  4458.000000  4458.000000  Default  30.0000  30.0000
133      place  4560.000000  4560.000000  Default  29.0000  29.0000
34        time  3302.000000  3302.000000  Default  28.0000  28.0000
310        try  1400.000000  1400.000000  Default  27.0000  27.0000
412       took   883.000000   883.000000  Default  26.0000  26.0000
..         ...          ...          ...      ...      ...      ...
92        nice   128.227325  1689.248901  Topic10  -5.8703  -0.2197
381     little   116.598427  1373.422119  Topic10  -5.9654  -0.1078
192    amazing   113.460754  1311.915039  Topic10  -5.9926  -0.0893
359  delicious   106.245964  1194.347168  Topic10  -6.0583  -0.0611
387      order   103.857544  1449.446533  Topic10  -6.0811  -0.2774

[866 rows x 6 columns], token_table=       Topic      Freq        Term
term                              
2925       2  0.127251   accepting
2925       4  0.127251   accepting
2925       6  0.254503   accepting
2925       7  0.127251   accepting
2925       8  0.127251   accepting
...      ...       ...         ...
13097      9  0.097819         yuk
13097     10  0.260852         yuk
21303      7  0.617682  とても気に入りました
25705      4  0.323379        台湾鸡排
21304      7  0.616917    探したお店でした

[3036 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 8, 2, 10, 1, 3, 5, 7, 9, 6])

## Stretch Goals

Complete one of more of these to push your score towards a three: 
* Create more visualizations of the LDA results and provide written analysis
* Incorporate RandomizedSearchCV into docoument classification pipeline
* Incorporate named entity recognition into your analysis
* Compare vectorization methods in the classification section
* Analyze more (or all) of the yelp dataset - this one is v. hard. 
* Use a generator object on the reviews file - this would help you with the analyzing the whole dataset.
* Incorporate any of the other yelp dataset entities in your analysis (business, users, etc.)